In [1]:
import numpy as np
import pandas as pd
import gc
import warnings
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score, recall_score, f1_score
from datetime import timedelta
import time




In [33]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1234)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [34]:
pd.DataFrame(y_train).value_counts()

0
1    285
0    170
Name: count, dtype: int64

In [37]:
svc_model = SVC(kernel='rbf')
svc_model.fit(X_train, y_train)
svc_pred = svc_model.predict(X_test)

In [39]:
print('Train Accuracy', accuracy_score(y_train, svc_model.predict(X_train)))
print('Test Accuracy', accuracy_score(y_test, svc_model.predict(X_test)))

print('\n')

print('Train Recall Score', recall_score(y_train, svc_model.predict(X_train)))
print('Test Recall Score', recall_score(y_test, svc_model.predict(X_test)))

print('\n')

print('Train F1 Score', f1_score(y_train, svc_model.predict(X_train)))
print('Test F1 Score', f1_score(y_test, svc_model.predict(X_test)))

Train Accuracy 0.989010989010989
Test Accuracy 0.9824561403508771


Train Recall Score 1.0
Test Recall Score 1.0


Train F1 Score 0.991304347826087
Test F1 Score 0.9863013698630136


## Grid Search

In [79]:
from sklearn.model_selection import GridSearchCV

param_space_grid = {
    'C': [200000, 260000, 270000], 
    'gamma': [1,3,5,7], 
    'degree': [2,4,6,8,10], 
    'kernel':  ['linear', 'poly', 'rbf']
}

estimator = SVC()
grid_search = GridSearchCV(estimator, param_grid= param_space_grid, scoring='accuracy', cv=3)
grid_search.fit(X_train, y_train)
print('best params: ', grid_search.best_params_)
print('val. score: %s' % random_search.best_score_)
print('test score: %s' % random_search.score(X_test, y_test))


best params:  {'C': 200000, 'degree': 2, 'gamma': 1, 'kernel': 'linear'}
val. score: 0.956043956043956
test score: 0.9736842105263158


## Randomized Search

In [80]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform, randint

param_space_random = {
    'C': loguniform(1e-6, 1e6),
    'gamma': loguniform(1e-6, 1e1),
    'degree': randint(1, 9),  # upper bound is exclusive
    'kernel': ['linear','poly','rbf']
}

estimator = SVC()
random_search = RandomizedSearchCV(estimator, param_space_random)
random_search.fit(X_train, y_train)
print('best params: ', random_search.best_params_)
print('val. score: %s' % random_search.best_score_)
print('test score: %s' % random_search.score(X_test, y_test))

best params:  {'C': 15338.225715091483, 'degree': 5, 'gamma': 2.764911971321577e-06, 'kernel': 'rbf'}
val. score: 0.9758241758241759
test score: 0.9824561403508771


## Bayesion Optimization using Skopt

In [67]:
from skopt import BayesSearchCV
param_space_skopt = {
    'C': (1e-6, 1e+6, 'log-uniform'),
    'gamma': (1e-6, 1e+1, 'log-uniform'),
    'degree': (1, 8),
    'kernel': ['linear', 'poly', 'rbf'],
}

opt = BayesSearchCV(
    estimator=SVC(),
    search_spaces=param_space_skopt,
    n_iter=32,
    cv=3
    
)

opt.fit(X_train, y_train)
print('val. score: %s' % opt.best_score_)
print('test score: %s' % opt.score(X_test, y_test))

val. score: 0.9713895666318114
test score: 0.9649122807017544


In [41]:
best_params = opt.best_params_
print('Best Parameters:', best_params)

Best Parameters: OrderedDict({'C': 0.020117103584690255, 'degree': 1, 'gamma': 2.6498956844158603, 'kernel': 'poly'})


In [42]:
# Create an SVM classifier with the best parameters
best_svc_model = SVC(**best_params)

# Fit the classifier on the training data
best_svc_model.fit(X_train, y_train)

# Predict on the test data
best_svc_pred = best_svc_model.predict(X_test)

# Evaluate the performance of the model
print('Train Accuracy with best parameters:', accuracy_score(y_train, best_svc_model.predict(X_train)))
print('Test Accuracy with best parameters:', accuracy_score(y_test, best_svc_pred))

print('\n')

print('Train Recall Score with best parameters:', recall_score(y_train, best_svc_model.predict(X_train)))
print('Test Recall Score with best parameters:', recall_score(y_test, best_svc_pred))

print('\n')

print('Train F1 Score with best parameters:', f1_score(y_train, best_svc_model.predict(X_train)))
print('Test F1 Score with best parameters:', f1_score(y_test, best_svc_pred))

Train Accuracy with best parameters: 0.9824175824175824
Test Accuracy with best parameters: 0.9736842105263158


Train Recall Score with best parameters: 0.9929824561403509
Test Recall Score with best parameters: 1.0


Train F1 Score with best parameters: 0.9860627177700348
Test F1 Score with best parameters: 0.9795918367346939


## Bayesion Optimization

In [81]:
from bayes_opt import BayesianOptimization


kernel_map = ['linear', 'poly', 'rbf']
param_space_bayes = {
    'C': (1e-6, 1e+6),
    'gamma': (1e-6, 1e+1),
    'degree': (1, 8),
    'kernel_idx': (0,2), # integer range for kernel
}

def svm_cv(C, gamma, degree, kernel_idx):
    kernel_idx = int(round(kernel_idx))  # round and cast to int
    kernel = kernel_map[kernel_idx]      # map to actual kernel name
    params = {
        'C': C,
        'gamma': gamma,
        'degree': int(round(degree)),    # round to nearest int
        'kernel': kernel,
    }
    estimator = SVC(**params)
    print(estimator)
    scores = cross_val_score(estimator, X_train, y_train, scoring='accuracy', cv=5).mean()
    score = scores.mean()
    return score

svm_bo = BayesianOptimization(f=svm_cv, pbounds=param_space_bayes)
svm_bo.maximize(init_points=20, n_iter=4)
results = pd.DataFrame(svm_bo.res)
results.sort_values(by='target', ascending=False, inplace=True)

|   iter    |  target   |     C     |  degree   |   gamma   | kernel... |
-------------------------------------------------------------------------
SVC(C=760746.9034517539, degree=7, gamma=8.076117722441444, kernel='linear')
| 1         | 0.9473    | 7.607e+05 | 7.334     | 8.076     | 0.4824    |
SVC(C=898963.5301324857, degree=1, gamma=8.561965998728557, kernel='poly')
| 2         | 0.9473    | 8.99e+05  | 1.459     | 8.562     | 1.467     |
SVC(C=777891.4209226471, degree=4, gamma=8.447135143146943, kernel='poly')
| 3         | 0.7846    | 7.779e+05 | 4.038     | 8.447     | 1.456     |
SVC(C=651707.8416858877, degree=7, gamma=2.234851188792184, kernel='poly')
| 4         | 0.8989    | 6.517e+05 | 7.415     | 2.235     | 0.5538    |
SVC(C=405298.30014620844, gamma=0.13956770007958524, kernel='poly')
| 5         | 0.9451    | 4.053e+05 | 2.734     | 0.1396    | 1.103     |
SVC(C=994089.0848444282, gamma=1.3780895069138792, kernel='poly')
| 6         | 0.9451    | 9.941e+05 | 3.109   

In [83]:
results

,target,params
0,0.947253,"{'C': 760746.9034517539, 'degree': 7.334238768..."
6,0.947253,"{'C': 134858.1595518323, 'degree': 6.952319464..."
20,0.947253,"{'C': 147331.23483004197, 'degree': 2.16992232..."
16,0.947253,"{'C': 147331.67556446703, 'degree': 3.60839267..."
1,0.947253,"{'C': 898963.5301324857, 'degree': 1.458795680..."
4,0.945055,"{'C': 405298.30014620844, 'degree': 2.73381526..."
5,0.945055,"{'C': 994089.0848444282, 'degree': 3.108661352..."
15,0.945055,"{'C': 726542.3168825186, 'degree': 3.325387998..."
7,0.940659,"{'C': 613281.3086717117, 'degree': 2.930195048..."
8,0.925275,"{'C': 607680.2316864919, 'degree': 5.062777205..."


In [69]:
best_result = svm_bo.max
params = best_result['params']
kernel_idx = int(round(params['kernel_idx']))
kernel = kernel_map[kernel_idx]
best_params = {
    'C': params['C'],
    'gamma': params['gamma'],
    'degree': int(round(params['degree'])),
    'kernel': kernel
}

print("Best parameters (decoded):", best_params)
print("Best accuracy score:", best_result['target'])

Best parameters (decoded): {'C': 262533.2538439819, 'gamma': 3.0003583119474135, 'degree': 8, 'kernel': 'linear'}
Best accuracy score: 0.9472527472527472


## Hyperopt

In [84]:
from hyperopt import hp
from hyperopt import fmin, tpe, Trials, STATUS_OK

param_space_hyperopt = {
    'C': hp.loguniform('C', np.log(1e-6), np.log(1e6)),
    'gamma': hp.loguniform('gamma', np.log(1e-6), np.log(1e1)),
    'degree': hp.quniform('degree', 1, 8, 1),
    'kernel': hp.choice('kernel', ['linear', 'poly', 'rbf']),
}

def objective(params):
    params['degree'] = int(params['degree'])  # cast degree to int
    model = SVC(**params)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()
    return {'loss': -score, 'status': STATUS_OK}

trials = Trials()
best = fmin(
    fn=objective,
    space=param_space_hyperopt,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.default_rng(42)
)

print("Best parameters:", best)

100%|█████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 21.50trial/s, best loss: -0.9714285714285715]
Best parameters: {'C': 0.13786321876846389, 'degree': 1.0, 'gamma': 6.685029500158167e-05, 'kernel': 0}
